In [2]:
from sklearn.datasets import fetch_openml   # Fetch dataset from openml

In [3]:
mnist = fetch_openml("mnist_784", version=1, cache=True, as_frame=False)# mnist 데이터 가져오기. 28 * 28 = 784개의 속성 

In [4]:
# 학습데이터, 타겟 데이터 
X = mnist.data
y = mnist.target

In [5]:
import torch   # pytorch 
from torch.utils.data import TensorDataset, DataLoader  # data loading utility
from sklearn.model_selection import train_test_split  # Split arrays or matrices into random train and test subsets

In [6]:
# proportion of the dataset to include in the train split : 1/7 , random_state: None 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=1/7, random_state=0)
# multi-dimensional matrix containing elements of a single data type
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
# 64-bit integer (signed)
y_train = torch.LongTensor(list(map(int, y_train)))
y_test = torch.LongTensor(list(map(int, y_test)))

In [7]:
import torch.nn as nn  # neural network 
import torch.nn.functional as F # dropout, relu 
from torch import optim # optimization algorithms
from torch.autograd import Variable # automatic differentiation

In [8]:
X_train = X_train.view(-1, 1, 28,28).float() # reshape
X_test = X_test.view(-1, 1, 28,28).float()  # reshape 
print(X_train.shape)
print(X_test.shape)

torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])


In [9]:
train = TensorDataset(X_train, y_train)  # training data
test = TensorDataset(X_test, y_test)  # test data 
BATCH_SIZE = 64
loader_train = DataLoader(train, batch_size= BATCH_SIZE, shuffle=False) # batch size = 64, no shuffle 
loader_test = DataLoader(test, batch_size= BATCH_SIZE, shuffle=False)  # batch size = 64, no shuffle 


In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # nn.Conv2d의 첫 두 파라미터는 입력 채널수(in_channels)와 출력 채널수(out_channels)
        self.conv1 = nn.Conv2d(1,10,kernel_size=5) # 10개의 특징맵  kernel size : 5 x 5 
        self.conv2 = nn.Conv2d(10,20,kernel_size=5) # 10개의 특징맵에서 20개의  특징맵. 
         
        self.conv2_drop = nn.Dropout2d() # 컨볼루션 결과 출력값 드롭아웃
        self.fc1 = nn.Linear(320, 50) # 컨볼루션과 드롭아웃을 거친 이미지는 nn.Linear의 일반 신경망을 거침
        self.fc2 = nn.Linear(50, 10) # 입력크기 50, 출력은 분류할 클래스 개수인 10으로 설정

        self.loss_fn = nn.CrossEntropyLoss() # loss function. CrossEntropy
        self.optimizer = optim.Adam(self.parameters(), lr=0.01) # adam optimizer. learning rate : 0.01

    def forward(self, x):  # forward function 
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        # 입력이이 첫 컨볼루션 계층을 거치고 F.max_pool2d함수를 거침 
        # F.max_pool2d의 두 번째 입력은 커널 크기
        # convolution이 max pooling을 통과한 x는 F.relu()활성화 함수를 거침침
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2)) # 두번째 컨볼루션 계층도 똑같이 반복
        x = x.view(-1, 320) # 컨볼루션 계층 2개를 거쳐 특징맵이 된 x를 1차원으로. (-1은 남는차원 모두, 320은 x가 가진 원소개수)
        x = F.relu(self.fc1(x)) 
        x = F.dropout(x, training=self.training) # ReLU 활성화 함수를 거친 뒤 드롭아웃을 사용
        x = self.fc2(x) # 0부터 9까지 레이블을 갖는 10개의 출력값을 가지는 신경망
        return F.log_softmax(x, dim=1) # return log softmax 

    

In [11]:
def fit(model, loader_train):
    optimizer = torch.optim.Adam(model.parameters()) # use Adam optimizer 
    error = nn.CrossEntropyLoss() # use Cross entropy 
    EPOCHS = 40 # 에포크 
    model.train() # 모델 학습 
    for epoch in range(EPOCHS): # EPOCHS 만큼 반복 
        correct = 0 # 초기화 
        for batch_idx, (X_batch, y_batch) in enumerate(loader_train): # loader_train 순회 
            var_X_batch = Variable(X_batch).float() # PyTorch Tensor의 Wrapper
            var_y_batch = Variable(y_batch)  # PyTorch Tensor의 Wrapper
            optimizer.zero_grad() # 그레디언트 초기화 
            output = model(var_X_batch)  # 모델 적용 
            loss = error(output, var_y_batch) # 손실
            loss.backward() # 손실 역전파 
            optimizer.step() # 역전파 단계에서 수집된 변화도로 매개변수를 조정
            predicted = torch.max(output.data, 1)[1]   #  return max_indices
            correct += (predicted == var_y_batch).sum() # correct count 
            if batch_idx % 200 == 0:
                print('에포크 : {} [{}/{} ({:.0f}%)]\t 손실함수 : {:.6f}\t Accuracy:{:.3f}%'\
                      .format(epoch, batch_idx*len(X_batch),len(loader_train),\
                        100.*batch_idx / len(loader_train),loss.data,correct*100./ (BATCH_SIZE*(batch_idx + 1))))


In [12]:
def evaluate(model): # 모델 평가 
    correct = 0
    for test_imgs, test_labels in loader_test:
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        # 가장 높은 값을 가진 인덱스:  예측값
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("테스트 데이터 정확도: {:.3f}% ".format(float(correct) / (len(loader_test)*BATCH_SIZE)))

cnn = CNN()
evaluate(cnn)
fit(cnn, loader_train)
cnn.eval()
evaluate(cnn)
index = 10
data = X_test[index].view(-1, 1, 28, 28).float()
output = cnn(data)
print('{} 번째 학습데이터의 테스트 결과 : {}'.format(index,output))
_, predicted = torch.max(output, 1)
print('{} 번째 데이터의 예측측 : {}'.format(index, predicted.numpy()))
print('실제 레이블 : {}'.format(y_test[index]))

테스트 데이터 정확도: 0.095% 
에포크 : 0 [0/938 (0%)]	 손실함수 : 18.919416	 Accuracy:7.812%
에포크 : 0 [12800/938 (21%)]	 손실함수 : 0.907760	 Accuracy:46.284%
에포크 : 0 [25600/938 (43%)]	 손실함수 : 0.800652	 Accuracy:61.943%
에포크 : 0 [38400/938 (64%)]	 손실함수 : 0.635431	 Accuracy:69.340%
에포크 : 0 [51200/938 (85%)]	 손실함수 : 0.570815	 Accuracy:73.551%
에포크 : 1 [0/938 (0%)]	 손실함수 : 0.135009	 Accuracy:95.312%
에포크 : 1 [12800/938 (21%)]	 손실함수 : 0.406122	 Accuracy:89.342%
에포크 : 1 [25600/938 (43%)]	 손실함수 : 0.295560	 Accuracy:89.760%
에포크 : 1 [38400/938 (64%)]	 손실함수 : 0.169738	 Accuracy:90.183%
에포크 : 1 [51200/938 (85%)]	 손실함수 : 0.296769	 Accuracy:90.475%
에포크 : 2 [0/938 (0%)]	 손실함수 : 0.141680	 Accuracy:93.750%
에포크 : 2 [12800/938 (21%)]	 손실함수 : 0.455063	 Accuracy:92.498%
에포크 : 2 [25600/938 (43%)]	 손실함수 : 0.428901	 Accuracy:92.511%
에포크 : 2 [38400/938 (64%)]	 손실함수 : 0.208003	 Accuracy:92.614%
에포크 : 2 [51200/938 (85%)]	 손실함수 : 0.143668	 Accuracy:92.712%
에포크 : 3 [0/938 (0%)]	 손실함수 : 0.189170	 Accuracy:95.312%
에포크 : 3 [12800/938 (21%